# 统计计算

## 分布

一组值及其对应的概率，例如骰子的1~6，及其对应的1/6（骰子质地均匀）；

## 曲奇饼问题 - Pmf解决

问题：有两个碗，碗1中有30个香草曲奇，10个巧克力曲奇，碗2中各有20个，问随便拿一个曲奇，从碗1取到香草的概率是多少？

In [15]:
# 可能1：碗1，可能2：碗2
from code.thinkbayes import Pmf
pmf = Pmf() # 创建概率质量函数对象
pmf.Set('Bowl 1',0.5) # 创建可能1，先验概率为0.5
pmf.Set('Bowl 2',0.5) # 创建可能2，先验概率为0.5
pmf.Mult('Bowl 1',0.75) # 可能1乘以似然度0.75
pmf.Mult('Bowl 2',0.5) # 可能1乘以似然度0.5
pmf.Normalize()
pmf.Prob('Bowl 1')

0.6000000000000001

## M&M问题 - Pmf解决

问题：有两袋M&M豆，94袋中30%褐色，20%黄色，20%红色，10%绿色，10%橙色，10%黄褐色，96袋中24%蓝色，20%绿色，16%橙色，14%黄色，13%红色，13%褐色，从两袋中各取一个豆，一个是黄色，一个是绿色，问黄色豆来自94年的袋子的概率是多少？

In [16]:
# 可能1：黄色来自94，绿色来自96，可能2：黄色来自96，绿色来自94
from code.thinkbayes import Pmf
pmf = Pmf() # 创建概率质量函数对象
pmf.Set('94,96',0.5) # 创建可能1，先验概率为0.5
pmf.Set('96,94',0.5) # 创建可能2，先验概率为0.5
pmf.Mult('94,96',0.2*0.2) # 可能1乘以似然度
pmf.Mult('96,94',0.14*0.1) # 可能1乘以似然度
pmf.Normalize()
pmf.Prob('94,96')

0.7407407407407408

## Monty Hall问题 - Pmf解决

蒙梯大厅问题：三扇门，其中一扇后有奖品，你随便挑选一扇，然后会展示剩下两扇中一扇没有奖品的门，然后你再做出选择，是保持原来的选择，还是从新选择另一扇没被打开的门；

In [17]:
# 选手选A，Monty打开门B且奖品不在B
# 可能1：奖品在A，可能2：奖品在B，可能3：奖品在C
from code.thinkbayes import Pmf
pmf = Pmf() # 创建概率质量函数对象
pmf.Set('A',1./3) # 创建可能1，先验概率为1/3
pmf.Set('B',1./3) # 创建可能2，先验概率为1/3
pmf.Set('C',1./3) # 创建可能3，先验概率为1/3
pmf.Mult('A',1./2) # 可能1乘以似然度
pmf.Mult('B',0) # 可能1乘以似然度
pmf.Mult('C',1) # 可能1乘以似然度
pmf.Normalize()
pmf.Prob('C') # 即选手改变自己的选择(由于B是被Monty打开的，因此选手只能改为C)的话选中概率

0.6666666666666666

## 贝叶斯框架 -- 曲奇饼问题

该框架的好处：
1. 很多问题可以改改就通用了，后面会看到；
2. 能够推广到拿多个饼，各个口味的概率；

### 构建Cookie类，基于Pmf类

In [18]:
class Cookie(Pmf):
    mixes = {
        'Bowl 1':dict(vanilla=0.75, chocolate=0.25),
        'Bowl 2':dict(vanilla=0.5, chocolate=0.5),
    }
    
    def __init__(self, hypos):
        '''
        Cookie类构造函数
        
        Args:
            hypos -- 全部假设
        '''
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()
        
    def Likelihood(self, data, hypo):
        '''
        根据传入data（此处是口味）求似然度
        
        Args:
            data -- 传入的信息，此处是口味
            hypo -- 某一种假设
            
        Returns:
            like -- 更新后的概率
        '''
        mix = self.mixes[hypo]
        like = mix[data]
        return like
        
    def Update(self, data):
        '''
        修正相应假设的概率
        
        Args:
            data -- 用于修正响应假设概率的信息，此处就是饼干口味
        '''
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()
        
    def PrintPredict(self):
        '''
        打印各个假设及其对应的概率
        '''
        for hypo, prob in self.Items():
            print hypo + ':' + str(prob)
        

### 测试Cookie类

#### 原始概率 -- 即先验概率，构造中定义了都是1

In [19]:
cookie = Cookie(['Bowl 1', 'Bowl 2'])
cookie.PrintPredict()

Bowl 2:0.5
Bowl 1:0.5


#### 更新一个香草信息

In [20]:
cookie.Update('vanilla')
cookie.PrintPredict()

Bowl 2:0.4
Bowl 1:0.6


#### 更新一个巧克力

In [21]:
cookie.Update('chocolate')
cookie.PrintPredict()

Bowl 2:0.571428571429
Bowl 1:0.428571428571


## 贝叶斯框架 -- Monty Hall问题

### 构建MontyHall类，继承自Pmf类

In [53]:
class MontyHall(Pmf):
    '''
    假设用户最开始选的是门A
    '''
    
    choose = 'A'
    
    def __init__(self, hypos):
        '''
        MontyHall类构造函数
        
        Args:
            hypos -- 全部假设
        '''
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()
        
    def Likelihood(self, data, hypo):
        '''
        根据传入data（此处是主持人去掉某个门）求似然度
        
        Args:
            data -- 传入的信息，此处是去掉的某扇门
            hypo -- 某一种假设
            
        Returns:
            like -- 更新后的概率
        '''
        return 0 if data == hypo else (0.5 if hypo == self.choose else 1)
        
    def Update(self, data):
        '''
        修正相应假设的概率，这个方法是叠加的
        
        Args:
            data -- 用于修正响应假设概率的信息，此处就是某扇门
        '''
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()
        
    def PrintPredict(self):
        '''
        打印各个假设及其对应的概率
        '''
        for hypo, prob in self.Items():
            print hypo + ':' + str(prob)
        

### 测试MontyHall类

#### 原始概率

In [51]:
montyhall = MontyHall(['A', 'B', 'C'])
montyhall.PrintPredict()

A:0.333333333333
C:0.333333333333
B:0.333333333333


#### 加入主持人信息

In [47]:
montyhall.Update('B')
montyhall.PrintPredict()

A:0.333333333333
C:0.666666666667
B:0.0


In [52]:
montyhall.Update('C')
montyhall.PrintPredict()

A:0.333333333333
C:0.0
B:0.666666666667


## 框架抽象封装

### Suite框架抽象 -- 只需实现Likelihood方法即可使用

In [65]:
from abc import ABCMeta, abstractmethod
class Suite(Pmf): # 抽象类
    
    __metaclass__ = ABCMeta
    
    def __init__(self, hypos):
        '''
        构造函数
        
        Args:
            hypos -- 全部假设
        '''
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()
    
    @abstractmethod  ##抽象方法
    def Likelihood(self, data, hypo):
        '''
        根据传入data求似然度
        
        Args:
            data -- 传入的信息
            hypo -- 某一种假设
            
        Returns:
            like -- 更新后的概率
        '''
        
    def Update(self, data):
        '''
        修正相应假设的概率，这个方法是叠加的
        
        Args:
            data -- 用于修正响应假设概率的信息
        '''
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()
        
    def PrintPredict(self):
        '''
        打印各个假设及其对应的概率
        '''
        for hypo, prob in self.Items():
            print hypo + ':' + str(prob)
        

### Monty问题在Suite框架中使用

In [66]:
class Monty(Suite):
    def Likelihood(self, data, hypo):
        return 0 if data == hypo else (0.5 if hypo == 'A' else 1)

monty = Monty('ABC')
monty.Update('B')
monty.PrintPredict()

A:0.333333333333
C:0.666666666667
B:0.0


### M&M豆问题在Suite框架中使用

#### 自己实现

In [72]:
class M_M(Suite):
    '''
    94袋中30%褐色，20%黄色，20%红色，10%绿色，10%橙色，10%黄褐色
    96袋中24%蓝色，20%绿色，16%橙色，14%黄色，13%红色，13%褐色
    从两袋中各取一个豆，一个是黄色，一个是绿色，问黄色豆来自94年的袋子的概率是多少？
    '''
    
    mixes = {
        '94':dict(hese=0.3,huangse=0.2,hongse=0.2,lvse=0.1,chengse=0.1,huanghese=0.1),
        '96':dict(hese=0.13,huangse=0.14,hongse=0.13,lvse=0.2,chengse=0.16,lanse=0.24),
    }
    
    def Likelihood(self, data, hypo):
        bags = hypo.split(',')
        print bags
        like = 1
        for i in range(len(data)):
            like *= self.mixes[bags[i]][data[i]]
        return like

m_m = M_M(['94,96,94','96,94,96'])
m_m.Update(['huangse', 'lvse', 'chengse'])
m_m.PrintPredict()

['94', '96', '94']
['96', '94', '96']
94,96,94:0.641025641026
96,94,96:0.358974358974


#### 书上实现

In [75]:
class M_M_B(Suite):
    mix94 = dict(hese=0.3,huangse=0.2,hongse=0.2,lvse=0.1,chengse=0.1,huanghese=0.1)
    mix96 = dict(hese=0.13,huangse=0.14,hongse=0.13,lvse=0.2,chengse=0.16,lanse=0.24)
    
    hypo1 = dict(bag1=mix94, bag2=mix96)
    hypo2 = dict(bag1=mix96, bag2=mix94)
    
    hypotheses = dict(A=hypo1, B=hypo2)
    
    def Likelihood(self, data, hypo):
        bag, color = data
        return self.hypotheses[hypo][bag][color]
        
m_m_b = M_M_B('AB')
m_m_b.Update(('bag1', 'huangse'))
m_m_b.Update(('bag2', 'lvse'))
m_m_b.PrintPredict()

A:0.740740740741
B:0.259259259259


PS：跟书本上的处理方法不太一致：
* 书本上：
    * 假设是A，B，也就是黄色豆来自94，或者95；
    * Update每次接受一组信息，包括第几次拿，以及颜色，比如('bag1', 'yellow')，表示第一次拿的黄色豆；
* 自己实现：
    * 假设是"94,96"和"96,94"；
    * Update是对应假设的一个集合，此处长度必须为2，对应"xx,yy"，['yellow','green']表示第一次拿黄色的，第二次是绿色；
    
区别：
* 书本上的实现方式允许无数次的迭代下去，可以拓展至任意次数的取豆子，并计算概率，自己实现的这个目前只支持与假设时放入的数量一致的信息个数，比如假设了"94,94"，那么Update只能提供[xx,yy]这种格式，否则报错
* 如果想要增加假设数量（比如拿了三次的话），书本上的虽然Update更简单，但是需要改动代码（因为它假设是写死在代码中的），而自己的写法是可以通过构造函数来修改的；

总体看自己实现的这个调用复杂，但是支持更多情况（比如想计算取3次，分别为黄色，绿色，橙色时的概率），而书本上的调用简单清晰，但是如果要支持取3次，且增加假设个数的话就需要修改类代码；

## 练习 -- 曲奇饼加强版

问题：有两个碗，碗1中有30个香草曲奇，10个巧克力曲奇，碗2中各有20个，问随便拿一个曲奇，从碗1取到香草的概率是多少？

增加：吃掉了取出的饼干，那么似然度就依赖于之前的取曲奇饼的行为；

### 普通曲奇饼问题Suite解决

In [81]:
class Cookie(Suite):
    mixes={
        'Bowl1':dict(vanilla=0.75, chocolate=0.25),
        'Bowl2':dict(vanilla=0.5, chocolate=0.5),
    }
    
    def Likelihood(self, data, hypo):
        return self.mixes[hypo][data]
        
cookie = Cookie(['Bowl1','Bowl2'])
cookie.Update('vanilla')
cookie.Update('chocolate')
cookie.Update('vanilla')
cookie.PrintPredict()

Bowl1:0.529411764706
Bowl2:0.470588235294


### 超级曲奇饼问题Suite解决 -- 拿出去的饼不放回

In [93]:
class SuperCookie(Suite):
    class Bowl:
        def __init__(self, vanilla=20, chocolate=20):
            self.vanilla=vanilla*1.
            self.chocolate=chocolate*1.

        def eat(self, cookie_name):
            if 'vanilla' == cookie_name:
                self.vanilla -= 1
            elif 'chocolate' == cookie_name:
                self.chocolate -= 1

        def check(self):
            return self.vanilla, self.chocolate
    
    hypo1 = dict(Bowl1=Bowl(30,10),Bowl2=Bowl(20,20))
    hypo2 = dict(Bowl1=Bowl(30,10),Bowl2=Bowl(20,20))
    hypotheses = dict(Bowl1=hypo1, Bowl2=hypo2)
    
    def Likelihood(self, data, hypo):
        _hypo = self.hypotheses[hypo]
        bowl = _hypo[hypo]
        vanilla, chocolate = bowl.check()
        print vanilla, chocolate
        bowl.eat(data)
        return vanilla/(vanilla+chocolate) if data=='vanilla' else chocolate/(vanilla+chocolate)
        
sc = SuperCookie(['Bowl1','Bowl2'])
sc.Update('vanilla')
sc.Update('chocolate')
sc.Update('vanilla')
sc.PrintPredict()

30.0 10.0
20.0 20.0
29.0 10.0
19.0 20.0
29.0 9.0
19.0 19.0
Bowl1:0.533742331288
Bowl2:0.466257668712


### 练习总结

重点：对于每个假设，分别一组碗供其在每次拿取饼干后，做减法（如果公用会有干扰），原本的拿取的概率计算由固定值，变为每次通过当前碗中的饼干实时计算得到；